In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'prepare/mesolitica-tpu.json'

In [2]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')

In [3]:
!rm -rf t5-small-bahasa
!mkdir t5-small-bahasa

In [4]:
blob = bucket.blob('t5-small/model.ckpt-510000.data-00000-of-00002')
blob.download_to_filename('t5-small-bahasa/model.ckpt-510000.data-00000-of-00002')

In [5]:
blob = bucket.blob('t5-small/model.ckpt-510000.data-00001-of-00002')
blob.download_to_filename('t5-small-bahasa/model.ckpt-510000.data-00001-of-00002')

In [6]:
blob = bucket.blob('t5-small/model.ckpt-510000.index')
blob.download_to_filename('t5-small-bahasa/model.ckpt-510000.index')

In [7]:
blob = bucket.blob('t5-small/model.ckpt-510000.meta')
blob.download_to_filename('t5-small-bahasa/model.ckpt-510000.meta')

In [8]:
blob = bucket.blob('t5-small/checkpoint')
blob.download_to_filename('t5-small-bahasa/checkpoint')

In [9]:
blob = bucket.blob('t5-small/operative_config.gin')
blob.download_to_filename('t5-small-bahasa/operative_config.gin')

In [10]:
# blob = bucket.blob('t5-base/events.out.tfevents.1589423125.general')
# blob.download_to_filename('t5-base-bahasa/events.out.tfevents.1589423125.general')

In [11]:
# !cp sp10m.cased.t5* t5-small-bahasa

In [12]:
# !pip3 install transformers -U
from transformers import T5Config, T5Model, load_tf_weights_in_t5

In [13]:
import os

out = 't5-small-bahasa-cased'
os.makedirs(out, exist_ok=True)

In [14]:
config = T5Config(
    vocab_size = 32128,
    n_positions=1024,
    d_ff = 2048,
    d_kv = 64,
    d_model = 512,
    dropout_rate = 0.1,
    inputs_length = 1024,
    num_heads = 8,
    num_layers = 6,
    decoder_start_token_id = 0,
    eos_token_id = 1,
    pad_token_id = 0)
print(config)
config.save_pretrained(out)

T5Config {
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "inputs_length": 1024,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 1024,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "vocab_size": 32128
}



In [16]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-small-bahasa/model.ckpt-510000')

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dropout(p=0.1, inplace

In [17]:
from transformers import CONFIG_NAME, WEIGHTS_NAME
CONFIG_NAME, WEIGHTS_NAME

('config.json', 'pytorch_model.bin')

In [18]:
import torch

torch.save(model.state_dict(), out + '/' + WEIGHTS_NAME)

In [19]:
from transformers import T5Config, T5Model, T5Tokenizer

In [20]:
tokenizer = T5Tokenizer('sp10m.cased.t5.model')
tokenizer.save_pretrained(out)

('t5-small-bahasa-cased/spiece.model',
 't5-small-bahasa-cased/special_tokens_map.json',
 't5-small-bahasa-cased/added_tokens.json')

In [21]:
tokenizer = T5Tokenizer.from_pretrained('./t5-small-bahasa-cased', lower = False)

In [22]:
config = T5Config.from_pretrained('./t5-small-bahasa-cased')

In [23]:
model = T5Model.from_pretrained('./t5-small-bahasa-cased/pytorch_model.bin', config = config)

In [24]:
model.save_pretrained(out)

In [25]:
# !transformers-cli upload ./t5-small-bahasa-cased

In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [27]:
tokenizer = T5Tokenizer.from_pretrained('./t5-small-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('./t5-small-bahasa-cased')

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at ./t5-small-bahasa-cased and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
input_ids = tokenizer.encode('soalan: gunung apakah yang paling tinggi didunia?', return_tensors = 'pt')
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

'Lobus'

In [29]:
# https://www.hmetro.com.my/mutakhir/2020/05/580438/peletakan-jawatan-tun-m-ditolak-bukan-lagi-isu
# original title, Peletakan jawatan Tun M ditolak, bukan lagi isu
string = 'PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu. Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin. Bekas Setiausaha Agung Bersatu Datuk Marzuki Yahya berkata, pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir. "Jadi ini agak berlawanan dengan keputusan yang kita sudah buat. Saya tak faham bagaimana Jabatan Pendaftar Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah buat keputusan di dalam mesyuarat, bukan seorang dua yang buat keputusan. "Semua keputusan mesti dibuat melalui parti. Walau apa juga perbincangan dibuat di luar daripada keputusan mesyuarat, ini bukan keputusan parti. "Apa locus standy yang ada pada Setiausaha Kerja untuk membawa perkara ini kepada JPPM. Seharusnya ia dibawa kepada Setiausaha Agung sebagai pentadbir kepada parti," katanya kepada Harian Metro. Beliau mengulas laporan media tempatan hari ini mengenai pengesahan JPPM bahawa Dr Mahathir tidak lagi menjadi Pengerusi Bersatu berikutan peletakan jawatannya di tengah-tengah pergolakan politik pada akhir Februari adalah sah. Laporan itu juga menyatakan, kedudukan Muhyiddin Yassin memangku jawatan itu juga sah. Menurutnya, memang betul Dr Mahathir menghantar surat peletakan jawatan, tetapi ditolak oleh MPT. "Fasal yang disebut itu terpakai sekiranya berhenti atau diberhentikan, tetapi ini mesyuarat sudah menolak," katanya. Marzuki turut mempersoal kenyataan media yang dibuat beberapa pimpinan parti itu hari ini yang menyatakan sokongan kepada Perikatan Nasional. "Kenyataan media bukanlah keputusan rasmi. Walaupun kita buat 1,000 kenyataan sekali pun ia tetap tidak merubah keputusan yang sudah dibuat di dalam mesyuarat. Kita catat di dalam minit apa yang berlaku di dalam mesyuarat," katanya.'
len(string.split())

282

In [30]:
input_ids = tokenizer.encode(f'tajuk: {string}', return_tensors = 'pt')
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

'Tidak timbul isu peletakan jawatan'

In [32]:
!tar -czvf t5-small-bahasa.gz t5-small-bahasa

t5-small-bahasa/
t5-small-bahasa/model.ckpt-510000.meta
t5-small-bahasa/model.ckpt-510000.index
t5-small-bahasa/checkpoint
t5-small-bahasa/model.ckpt-510000.data-00000-of-00002
t5-small-bahasa/model.ckpt-510000.data-00001-of-00002
t5-small-bahasa/operative_config.gin
